In [0]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import sys

import warnings
warnings.filterwarnings("ignore")

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

import numpy as np
import pandas as pd
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D

%matplotlib inline
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import io
import zipfile
from PIL import Image

Using TensorFlow backend.


In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
!ls

adience_split	   datalab		 inception_model.h5
adience_split.zip  inceptionHistoryDict


### **Loading Train Data**

In [5]:
file_list = drive.ListFile({'q': "'1IsZnTUo2pSXDH00Xq42Ye8JJe768XUtB' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: selfie_test_data.zip, id: 1Kx5Xgz1eN69c0OvgLPvWniXFdBmZ9gRi
title: adience_split.zip, id: 11GsvwXPQ4UGSSU2bILA_QeXXThHkFpy4
title: selfie_test_data, id: 1YcwE0YD_fFJxGvxPoQK7STEH58eGQXeZ
title: adience_split, id: 13HFhSYmArdZXIcf9jN78crzxFHl8Rxdt


In [0]:
data_downloaded = drive.CreateFile({'id': '11GsvwXPQ4UGSSU2bILA_QeXXThHkFpy4'})
data_downloaded.GetContentFile('adience_split.zip')

In [0]:
data = zipfile.ZipFile('adience_split.zip', 'r')
dir_list = data.namelist()

In [0]:
data.extractall('adience_split/')

In [9]:
!ls

adience_split	   datalab		 inception_model.h5
adience_split.zip  inceptionHistoryDict


### Model

In [0]:
from keras.engine import  Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import SGD, Adam

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions

In [11]:
#custom parameters
nb_class = 2
hidden_dim = 512

inception_model = InceptionV3(weights='imagenet', include_top=True)
last_layer = inception_model.get_layer('avg_pool').output
# x = Dense(hidden_dim, activation='relu', name='my_dense')(last_layer)
out = Dense(nb_class, activation='softmax', name='my_output')(last_layer)

layer_count = 0
for layer in inception_model.layers: layer_count = layer_count+1
for l in range(layer_count-1): inception_model.layers[l].trainable=False
  
inception_model = Model(inception_model.input, out)
inception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

batch_normalization_87 (BatchNo (None, None, None, 3 1152        conv2d_87[0][0]                  
__________________________________________________________________________________________________
batch_normalization_91 (BatchNo (None, None, None, 3 1152        conv2d_91[0][0]                  
__________________________________________________________________________________________________
activation_87 (Activation)      (None, None, None, 3 0           batch_normalization_87[0][0]     
__________________________________________________________________________________________________
activation_91 (Activation)      (None, None, None, 3 0           batch_normalization_91[0][0]     
__________________________________________________________________________________________________
conv2d_88 (Conv2D)              (None, None, None, 3 442368      activation_87[0][0]              
__________________________________________________________________________________________________
conv2d_89 

### Training on Entire Dataset

In [12]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
source_path = 'adience_split/adience_split/'

train_generator = datagen.flow_from_directory(
        source_path+'train',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

val_generator = datagen.flow_from_directory(
        source_path+'val',
        target_size=(224, 224),
        batch_size=32, 
        shuffle = True)
#         class_mode='binary')

test_generator = datagen.flow_from_directory(
        source_path+'test',
        target_size=(224, 224),
        batch_size=32,
        shuffle = True)
#         class_mode='binary')

Found 8950 images belonging to 2 classes.
Found 3837 images belonging to 2 classes.
Found 5481 images belonging to 2 classes.


In [0]:
# training second time
history = inception_model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=50,
        validation_data=val_generator,
        validation_steps=80,
        shuffle = True)

Epoch 1/50
100/100 [==============================] - 96s 955ms/step - loss: 0.6329 - acc: 0.6297 - val_loss: 0.6879 - val_acc: 0.6609
Epoch 2/50
100/100 [==============================] - 81s 812ms/step - loss: 0.5629 - acc: 0.7019 - val_loss: 0.6774 - val_acc: 0.6770
Epoch 3/50
100/100 [==============================] - 79s 790ms/step - loss: 0.5376 - acc: 0.7183 - val_loss: 0.5536 - val_acc: 0.7262
Epoch 4/50
 60/100 [=================>............] - ETA: 14s - loss: 0.5061 - acc: 0.7396

100/100 [==============================] - 76s 762ms/step - loss: 0.5019 - acc: 0.7450 - val_loss: 0.4984 - val_acc: 0.7703
Epoch 5/50
100/100 [==============================] - 76s 758ms/step - loss: 0.5192 - acc: 0.7303 - val_loss: 0.5204 - val_acc: 0.7559
Epoch 6/50
100/100 [==============================] - 75s 750ms/step - loss: 0.4960 - acc: 0.7580 - val_loss: 0.5528 - val_acc: 0.7414
Epoch 7/50
 81/100 [=======================>......] - ETA: 7s - loss: 0.4888 - acc: 0.7546

100/100 [==============================] - 75s 749ms/step - loss: 0.4932 - acc: 0.7537 - val_loss: 0.5049 - val_acc: 0.7676
Epoch 8/50
100/100 [==============================] - 75s 752ms/step - loss: 0.4900 - acc: 0.7563 - val_loss: 0.5853 - val_acc: 0.7348
Epoch 9/50
100/100 [==============================] - 76s 764ms/step - loss: 0.4635 - acc: 0.7754 - val_loss: 0.6805 - val_acc: 0.7063
Epoch 10/50
 81/100 [=======================>......] - ETA: 7s - loss: 0.5102 - acc: 0.7446

100/100 [==============================] - 77s 766ms/step - loss: 0.5128 - acc: 0.7456 - val_loss: 0.5283 - val_acc: 0.7684
Epoch 11/50
100/100 [==============================] - 77s 773ms/step - loss: 0.4839 - acc: 0.7644 - val_loss: 0.5304 - val_acc: 0.7574
Epoch 12/50
100/100 [==============================] - 76s 763ms/step - loss: 0.4633 - acc: 0.7796 - val_loss: 0.6147 - val_acc: 0.7211
Epoch 13/50
 76/100 [=====================>........] - ETA: 9s - loss: 0.4658 - acc: 0.7685

100/100 [==============================] - 77s 768ms/step - loss: 0.4669 - acc: 0.7681 - val_loss: 0.7591 - val_acc: 0.6922
Epoch 14/50
100/100 [==============================] - 77s 773ms/step - loss: 0.4635 - acc: 0.7744 - val_loss: 0.5296 - val_acc: 0.7648
Epoch 15/50
100/100 [==============================] - 77s 775ms/step - loss: 0.4463 - acc: 0.7863 - val_loss: 0.6390 - val_acc: 0.7258
Epoch 16/50
 81/100 [=======================>......] - ETA: 7s - loss: 0.4893 - acc: 0.7627

...

100/100 [==============================] - 79s 792ms/step - loss: 0.4701 - acc: 0.7741 - val_loss: 0.5731 - val_acc: 0.7613
Epoch 32/50
100/100 [==============================] - 79s 793ms/step - loss: 0.4359 - acc: 0.7916 - val_loss: 0.5955 - val_acc: 0.7441
Epoch 33/50
100/100 [==============================] - 79s 792ms/step - loss: 0.4352 - acc: 0.7916 - val_loss: 0.6206 - val_acc: 0.7352
Epoch 34/50
 81/100 [=======================>......] - ETA: 7s - loss: 0.4261 - acc: 0.8005

100/100 [==============================] - 79s 791ms/step - loss: 0.4270 - acc: 0.7997 - val_loss: 0.6848 - val_acc: 0.7145
Epoch 35/50
100/100 [==============================] - 79s 791ms/step - loss: 0.4505 - acc: 0.7872 - val_loss: 0.5921 - val_acc: 0.7516
Epoch 36/50
100/100 [==============================] - 79s 790ms/step - loss: 0.4541 - acc: 0.7856 - val_loss: 0.5748 - val_acc: 0.7590
Epoch 37/50
 83/100 [=======================>......] - ETA: 6s - loss: 0.4496 - acc: 0.7876

100/100 [==============================] - 78s 783ms/step - loss: 0.4437 - acc: 0.7909 - val_loss: 0.6547 - val_acc: 0.7281
Epoch 38/50
100/100 [==============================] - 78s 779ms/step - loss: 0.4504 - acc: 0.7844 - val_loss: 0.6548 - val_acc: 0.7227
Epoch 39/50
100/100 [==============================] - 78s 780ms/step - loss: 0.4459 - acc: 0.7934 - val_loss: 1.0150 - val_acc: 0.6605
Epoch 40/50
 80/100 [=======================>......] - ETA: 7s - loss: 0.4471 - acc: 0.7831

100/100 [==============================] - 79s 790ms/step - loss: 0.4546 - acc: 0.7802 - val_loss: 0.7852 - val_acc: 0.6953
Epoch 41/50
100/100 [==============================] - 79s 792ms/step - loss: 0.4432 - acc: 0.7864 - val_loss: 0.6209 - val_acc: 0.7395
Epoch 42/50
100/100 [==============================] - 79s 790ms/step - loss: 0.4225 - acc: 0.8000 - val_loss: 0.5707 - val_acc: 0.7590
Epoch 43/50
 78/100 [======================>.......] - ETA: 8s - loss: 0.4246 - acc: 0.8017

100/100 [==============================] - 79s 792ms/step - loss: 0.4242 - acc: 0.8031 - val_loss: 0.5862 - val_acc: 0.7578
Epoch 44/50
 99/100 [============================>.] - ETA: 0s - loss: 0.4550 - acc: 0.7787

In [19]:
history = inception_model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=50,
        validation_data=val_generator,
        validation_steps=80,
        shuffle = True)

Epoch 1/50
100/100 [==============================] - 97s 967ms/step - loss: 0.7138 - acc: 0.6128 - val_loss: 0.8141 - val_acc: 0.5891
Epoch 2/50
100/100 [==============================] - 81s 808ms/step - loss: 0.5531 - acc: 0.7131 - val_loss: 0.5133 - val_acc: 0.7449
Epoch 3/50
100/100 [==============================] - 78s 777ms/step - loss: 0.5232 - acc: 0.7322 - val_loss: 0.5398 - val_acc: 0.7359
Epoch 4/50
 60/100 [=================>............] - ETA: 14s - loss: 0.5031 - acc: 0.7532

100/100 [==============================] - 77s 769ms/step - loss: 0.5049 - acc: 0.7504 - val_loss: 0.5636 - val_acc: 0.7285
Epoch 5/50
100/100 [==============================] - 76s 764ms/step - loss: 0.5032 - acc: 0.7469 - val_loss: 0.5435 - val_acc: 0.7391
Epoch 6/50
100/100 [==============================] - 77s 766ms/step - loss: 0.4841 - acc: 0.7609 - val_loss: 0.5700 - val_acc: 0.7363
Epoch 7/50
 79/100 [======================>.......] - ETA: 7s - loss: 0.5161 - acc: 0.7342

100/100 [==============================] - 76s 763ms/step - loss: 0.5078 - acc: 0.7409 - val_loss: 0.5806 - val_acc: 0.7344
Epoch 8/50
100/100 [==============================] - 76s 762ms/step - loss: 0.4800 - acc: 0.7641 - val_loss: 0.6063 - val_acc: 0.7199
Epoch 9/50
100/100 [==============================] - 76s 757ms/step - loss: 0.4714 - acc: 0.7681 - val_loss: 0.6313 - val_acc: 0.7176
Epoch 10/50
 84/100 [========================>.....] - ETA: 5s - loss: 0.4594 - acc: 0.7772

100/100 [==============================] - 76s 757ms/step - loss: 0.4614 - acc: 0.7747 - val_loss: 0.5793 - val_acc: 0.7445
Epoch 11/50
100/100 [==============================] - 76s 757ms/step - loss: 0.4739 - acc: 0.7713 - val_loss: 0.5632 - val_acc: 0.7523
Epoch 12/50
100/100 [==============================] - 76s 757ms/step - loss: 0.4660 - acc: 0.7762 - val_loss: 0.5895 - val_acc: 0.7465
Epoch 13/50
 83/100 [=======================>......] - ETA: 6s - loss: 0.4670 - acc: 0.7673

100/100 [==============================] - 75s 754ms/step - loss: 0.4574 - acc: 0.7744 - val_loss: 0.7371 - val_acc: 0.6992
Epoch 14/50
100/100 [==============================] - 75s 751ms/step - loss: 0.4699 - acc: 0.7709 - val_loss: 0.5566 - val_acc: 0.7566
Epoch 15/50
100/100 [==============================] - 75s 755ms/step - loss: 0.4616 - acc: 0.7728 - val_loss: 0.6364 - val_acc: 0.7246
Epoch 16/50
 83/100 [=======================>......] - ETA: 6s - loss: 0.4707 - acc: 0.7613

...

100/100 [==============================] - 75s 753ms/step - loss: 0.3904 - acc: 0.8150 - val_loss: 0.9181 - val_acc: 0.7016
Epoch 38/50
100/100 [==============================] - 75s 749ms/step - loss: 0.3981 - acc: 0.8167 - val_loss: 0.6799 - val_acc: 0.7457
Epoch 39/50
100/100 [==============================] - 75s 754ms/step - loss: 0.4037 - acc: 0.8056 - val_loss: 0.7744 - val_acc: 0.7156
Epoch 40/50
 82/100 [=======================>......] - ETA: 6s - loss: 0.4003 - acc: 0.8159

100/100 [==============================] - 75s 752ms/step - loss: 0.3913 - acc: 0.8200 - val_loss: 0.9463 - val_acc: 0.7086
Epoch 41/50
100/100 [==============================] - 75s 749ms/step - loss: 0.3926 - acc: 0.8138 - val_loss: 0.6301 - val_acc: 0.7551
Epoch 42/50
100/100 [==============================] - 75s 751ms/step - loss: 0.3892 - acc: 0.8137 - val_loss: 0.6611 - val_acc: 0.7477
Epoch 43/50
 81/100 [=======================>......] - ETA: 7s - loss: 0.3474 - acc: 0.8353

100/100 [==============================] - 75s 754ms/step - loss: 0.3573 - acc: 0.8296 - val_loss: 0.7402 - val_acc: 0.7332
Epoch 44/50
100/100 [==============================] - 75s 749ms/step - loss: 0.3889 - acc: 0.8156 - val_loss: 0.7442 - val_acc: 0.7414
Epoch 45/50
100/100 [==============================] - 75s 752ms/step - loss: 0.3790 - acc: 0.8287 - val_loss: 0.7313 - val_acc: 0.7488
Epoch 46/50
 82/100 [=======================>......] - ETA: 6s - loss: 0.3978 - acc: 0.8116

100/100 [==============================] - 75s 752ms/step - loss: 0.3975 - acc: 0.8127 - val_loss: 0.6605 - val_acc: 0.7617
Epoch 47/50
100/100 [==============================] - 75s 754ms/step - loss: 0.3796 - acc: 0.8197 - val_loss: 0.6734 - val_acc: 0.7430
Epoch 48/50
100/100 [==============================] - 75s 752ms/step - loss: 0.3858 - acc: 0.8266 - val_loss: 0.8530 - val_acc: 0.7078
Epoch 49/50
 82/100 [=======================>......] - ETA: 6s - loss: 0.3738 - acc: 0.8157

100/100 [==============================] - 75s 749ms/step - loss: 0.3728 - acc: 0.8198 - val_loss: 0.7503 - val_acc: 0.7312
Epoch 50/50
100/100 [==============================] - 75s 752ms/step - loss: 0.3554 - acc: 0.8406 - val_loss: 0.8370 - val_acc: 0.7285


In [0]:
inception_model.save('inception_model.h5')

In [0]:
import pickle

with open('inceptionHistoryDict', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [0]:
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [23]:
uploaded = drive.CreateFile({'title': 'inception_model.h5'})
uploaded.SetContentFile('inception_model.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1eaHYc9rJbcSLqrnejZWxRnRDP0MZYgSj


In [24]:
uploaded = drive.CreateFile({'title': 'inceptionHistoryDict'})
uploaded.SetContentFile('inceptionHistoryDict')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1kTirMRDEVzvC5OHpTpwO8s2pN7p0SMKd


In [0]:
custom_vgg_model.predict_generator(test_generator)

array([[4.5621413e-04, 9.9954373e-01],
       [1.0000000e+00, 2.2820428e-08],
       [9.9999988e-01, 7.3189206e-08],
       ...,
       [4.6054681e-23, 1.0000000e+00],
       [1.0000000e+00, 5.1509946e-13],
       [1.6056726e-12, 1.0000000e+00]], dtype=float32)

In [0]:
custom_vgg_model.evaluate_generator(train_generator, pickle_safe=True)

[0.00400182354513798, 0.9985474860335195]

In [0]:
custom_vgg_model.evaluate_generator(val_generator, pickle_safe=True)

[0.3491943102125901, 0.9478759446086938]

In [0]:
custom_vgg_model.evaluate_generator(test_generator, pickle_safe=True)

[0.4086071009749553, 0.9366903849771218]

In [0]:
ls

adience_split/  adience_split.zip  datalab/  vggf_train_full.h5  vggHistoryDict
